In [357]:
import os
import sys
import time
from pathlib import Path
module_path = os.path.abspath('../src/utils')
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath('../src/data_preprocessing')
sys.path.append(module_path)

import pandas as pd
import numpy as np
import ants

from base_mri import *
import mri_label
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Getting list of images

In [358]:
df_mprage = load_reference_table()
# df_mprage = transform_reference_table(df_mprage)

In [359]:
df_mprage

,IMAGE_DATA_ID,SUBJECT,GROUP,SEX,AGE,VISIT,DESCRIPTION,TYPE,ACQ_DATE,MACRO_GROUP,SUBJECT_IMAGE_ID
0,I254770,153_S_4172,AD,M,76,22,MT1; GradWarp; N3m,Processed,2011-08-17,AD,153_S_4172#I254770
1,I398400,153_S_4172,AD,M,78,30,MT1; GradWarp; N3m,Processed,2013-09-25,AD,153_S_4172#I398400
2,I297604,153_S_4172,AD,M,77,25,MT1; GradWarp; N3m,Processed,2012-03-28,AD,153_S_4172#I297604
3,I335938,153_S_4172,AD,M,77,28,MT1; GradWarp; N3m,Processed,2012-08-16,AD,153_S_4172#I335938
4,I272738,153_S_4172,AD,M,76,24,MT1; GradWarp; N3m,Processed,2011-10-31,AD,153_S_4172#I272738
...,...,...,...,...,...,...,...,...,...,...,...
12844,I474751,002_S_0729,MCI,F,68,8,HarP 135 final release 2015,Processed,2009-08-13,MCI,002_S_0729#I474751
12845,I124007,002_S_0729,MCI,F,67,6,MPR; ; N3,Processed,2008-09-29,MCI,002_S_0729#I124007
12846,I97014,002_S_0729,MCI,F,67,5,MPR; ; N3,Processed,2008-03-05,MCI,002_S_0729#I97014
12847,I97016,002_S_0729,MCI,F,67,5,MPR; ; N3,Processed,2008-03-05,MCI,002_S_0729#I97016


In [533]:
mri_input_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210402"
mri_output_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/train"
preprocessed_imgs_list,_,_ = list_available_images(mri_input_path,file_format=".nii.gz")

Looking for MRI images in path: /home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210402 

Found a total of  896  images.
Found a total of  0  mask images.
Available images to process:  896 



In [ ]:
# def rename_images_with_label(old_list,new_list):
#     for old_name,new_name in zip(old_list,new_list):
#         os.rename(old_name,new_name)

In [527]:
# new_list = [mri_input_path+"/ADNI_"+x.split("/")[-1]  for x in preprocessed_imgs_list]
# rename_images_with_label(preprocessed_imgs_list,new_list)

In [528]:
# old_mci = [x for x in preprocessed_imgs_list if '_MCI' in x]
# old_ad = [x for x in preprocessed_imgs_list if '_AD' in x]
# old_cn = [x for x in preprocessed_imgs_list if '_CN' in x]

# mci = [x.replace("_MCI",'') for x in preprocessed_imgs_list if '_MCI' in x]
# ad = [x.replace("_AD",'') for x in preprocessed_imgs_list if '_AD' in x]
# cn = [x.replace("_CN",'') for x in preprocessed_imgs_list if '_CN' in x]

In [532]:
# rename_images_with_label(old_mci,mci)

In [494]:
mri_input_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210402"
mri_output_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/train"
preprocessed_imgs_list,_,_ = list_available_images(mri_input_path,file_format=".nii.gz")

mri_processed_path = '/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/processed/20210329_coronal_50'
processed_imgs_list,_,_ = list_available_images(mri_processed_path,file_format=".npz")

Looking for MRI images in path: /home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210402 

Found a total of  896  images.
Found a total of  0  mask images.
Available images to process:  896 

Looking for MRI images in path: /home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/processed/20210329_coronal_50 

Found a total of  25776  images.
Found a total of  0  mask images.
Available images to process:  25776 



In [555]:
def create_reference_table(image_list,output_path,previous_reference_file_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/reference/MRI_MPRAGE.csv") -> None:
    
    '''
    Create a new reference dataframe based on the current list of images.

    Reference dataframe contains information about the patient and their Alzheimer's Disease current situation.
    
    Parameters
    ----------
    
    image_list: List containing the path and file name for the images to rename.
    
    output_path: Path to save the new the reference file. Should be the same path where the images are located
    
    previous_reference_file_path: Path to the previous reference file containing information about patients and their disease stage.
    
    Returns
    ----------

    Saves a reference dataframe along with the images to be further used in the training phase.

    '''
         
    df_original_reference = load_reference_table(path = previous_reference_file_path)
    image_references = create_image_references(image_list)

    df_image_path = pd.DataFrame(data = {
        'SUBJECT_IMAGE_ID':[x[0] for x in image_references], 
        'UNIQUE_IMAGE_ID':[x[1] for x in image_references], 
        'IMAGE_PATH':[x[2] for x in image_references]})
    
    df_reference = pd.merge(df_image_path,df_original_reference,how='inner',on='SUBJECT_IMAGE_ID')
    df_reference.to_csv(output_path + 'REFERENCE.csv',index=False)
    print("Reference file saved at: ",output_path)
    return df_reference

def create_image_references(imgs_list):
    "Creates a dictionary where the key is the image id and the value is the path to the image."
    imgs = []
    for path in imgs_list: 
        imgs.append(create_unique_patient_image_id(path))
    return imgs

def create_unique_patient_image_id(path):
    
    img_id = 'I'+path.split('_I')[-1].split('_')[0]
    if '.' in img_id:
        img_id =  img_id.split('.')[0]
    unique_image_id = 'I'+path.split('_I')[-1].split('.')[0]
    
    patient_splits = path.split('/')[-1].split('_')
    patient_id = patient_splits[1] + '_' + patient_splits[2] + '_' +patient_splits[3]

    unique_patient_image_id = patient_id + "#" + img_id
    
    return unique_patient_image_id, unique_image_id, path

In [556]:
output_path = '/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/processed/20210329_coronal_50/'
df = create_reference_table(processed_imgs_list,output_path,previous_reference_file_path = "/home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/reference/MRI_MPRAGE.csv")

Reference file saved at:  /home/lucasthim1/mmml-alzheimer-diagnosis/data/mri/processed/20210329_coronal_50/


Dataframes can be saved and then loaded to train the neural net. 
X = paths to load images
y = GROUP or MACRO_GROUP